# **Overall Evaluation Metric**

In questo notebook verranno implementati i passaggi per calcolare il risultato della valutazione finale rispetto ai 5 task implementati per vedere come ci saremmo posizionati nel ranking della competizione.

Di seguito ci sono le regole per l'assegnamento dei punteggi:

- Classificazione della condizione normale/anomala: 10 punti
- Per i dati correttamente rilevati come anomali, classificazione del tipo di anomalia (contaminazione da bolle / guasto alla valvola a solenoide / guasto sconosciuto): 10 punti
- Per i dati correttamente identificati come contaminazione da bolle, identificazione della posizione della bolla: 10 punti
- Per i dati correttamente identificati come guasto alla valvola a solenoide, identificazione della valvola guasta: 10 punti
- Per le valvole a solenoide correttamente identificate come guaste, previsione del rapporto di apertura:

	punteggio = max(-|valore reale – valore predetto| + 20, 0)
- Per lo Spacecraft-4, i punteggi sono raddoppiati, data la difficoltà.

### **1. Importazione delle predizioni complessive finali** (final_test_data_predictions.csv) **e del ground truth** (answer_competition.csv)

In [158]:
import pandas as pd
import numpy as np

# Carico le predizioni 
df_pred = pd.read_csv("dataset/outputs/final_test_data_predictions.csv")
df_pred['BinaryCondition'] = df_pred['Condition'].apply(lambda x: 0 if x == 0 else 1)
df_pred.head(10)


,Case,Condition,anomaly_location,fault_location,valve_opening,BinaryCondition
0,178,2,2,0,100,1
1,179,3,0,2,26,1
2,180,0,0,0,100,0
3,181,3,0,4,67,1
4,182,0,0,0,100,0
5,183,0,0,0,100,0
6,184,1,0,0,100,1
7,185,0,0,0,100,0
8,186,2,6,0,100,1
9,187,0,0,0,100,0


In [159]:
# Carico il ground truth
df_gt = pd.read_csv("dataset/test/answer_competition.csv")
df_gt.head(10)

,Spacecraft No.,ID,task1,task2,task3,task4,task5,Test condition
0,1,178,1,2,2,0,100,BP2 bubble anomaly
1,1,179,1,3,0,2,22,SV2 valve fault
2,1,180,0,0,0,0,100,Normal
3,1,181,1,3,0,4,76,SV4 valve fault
4,1,182,0,0,0,0,100,Normal
5,1,183,0,0,0,0,100,Normal
6,1,184,1,1,0,0,100,Unknown anomaly
7,1,185,0,0,0,0,100,Normal
8,1,186,1,2,6,0,100,BP6 bubble anomaly
9,1,187,0,0,0,0,100,Normal


In [160]:
# Merge tra i due dataframe su Case e ID
df_merged = pd.merge(df_pred, df_gt, left_on="Case", right_on="ID", how="inner")
df_merged= df_merged.drop(columns=["ID", "Test condition"])
df_merged.head(10)

,Case,Condition,anomaly_location,fault_location,valve_opening,BinaryCondition,Spacecraft No.,task1,task2,task3,task4,task5
0,178,2,2,0,100,1,1,1,2,2,0,100
1,179,3,0,2,26,1,1,1,3,0,2,22
2,180,0,0,0,100,0,1,0,0,0,0,100
3,181,3,0,4,67,1,1,1,3,0,4,76
4,182,0,0,0,100,0,1,0,0,0,0,100
5,183,0,0,0,100,0,1,0,0,0,0,100
6,184,1,0,0,100,1,1,1,1,0,0,100
7,185,0,0,0,100,0,1,0,0,0,0,100
8,186,2,6,0,100,1,1,1,2,6,0,100
9,187,0,0,0,100,0,1,0,0,0,0,100


Rinomino le colonne di df_merged per chiarezza 

In [161]:
df_merged.rename(columns={
    "BinaryCondition": "task1_predicted",
    "Condition": "task2_predicted",
    "anomaly_location": "task3_predicted",
    "fault_location": "task4_predicted",
    "valve_opening": "task5_predicted",
    "task1": "task1_expected",
    "task2": "task2_expected",
    "task3": "task3_expected",
    "task4": "task4_expected",
    "task5": "task5_expected",
}, inplace=True)
df_merged.head(5)

,Case,task2_predicted,task3_predicted,task4_predicted,task5_predicted,task1_predicted,Spacecraft No.,task1_expected,task2_expected,task3_expected,task4_expected,task5_expected
0,178,2,2,0,100,1,1,1,2,2,0,100
1,179,3,0,2,26,1,1,1,3,0,2,22
2,180,0,0,0,100,0,1,0,0,0,0,100
3,181,3,0,4,67,1,1,1,3,0,4,76
4,182,0,0,0,100,0,1,0,0,0,0,100


In [162]:
# Inizializzo punteggio totale
total_score = 0

### Task 1 evaluation

In [163]:
# --- Task 1: Normal vs Abnormal ---
max_task1_score = 0
correct_task1 = df_merged["task1_predicted"] == df_merged["task1_expected"]
df_merged["score_task1"] = correct_task1.astype(int) * 10
df_merged.loc[df_merged["Spacecraft No."] == 4, "score_task1"] *= 2
df_merged[["Case", "task1_predicted","task1_expected", "score_task1"]]


,Case,task1_predicted,task1_expected,score_task1
0,178,1,1,10
1,179,1,1,10
2,180,0,0,10
3,181,1,1,10
4,182,0,0,10
5,183,0,0,10
6,184,1,1,10
7,185,0,0,10
8,186,1,1,10
9,187,0,0,10


In [164]:
# Calcolo del punteggio massimo ottenibile per il Task 1
df_merged["max_score_task1"] = 10
df_merged.loc[df_merged["Spacecraft No."] == 4, "max_score_task1"] = 20
max_task1_score = int(df_merged["max_score_task1"].sum())

task1_score = int(df_merged["score_task1"].sum())

print("Punteggio ottenuto sul Task 1: ", task1_score)
print("Punteggio massimo sul Task 1: ", max_task1_score)

Punteggio ottenuto sul Task 1:  640
Punteggio massimo sul Task 1:  690


### Task 2 evaluation

In [165]:
# --- Task 2: Anomaly vs Fault vs Unknown (solo se task1 è 1) ---
mask_abnormal = df_merged["task1_predicted"] == 1
correct_task2 = (df_merged["task2_predicted"] == df_merged["task2_expected"]) & mask_abnormal
df_merged["score_task2"] = correct_task2.astype(int) * 10
df_merged.loc[df_merged["Spacecraft No."] == 4, "score_task2"] *= 2
df_merged[["Case", "task2_predicted","task2_expected", "score_task2"]]


,Case,task2_predicted,task2_expected,score_task2
0,178,2,2,10
1,179,3,3,10
2,180,0,0,0
3,181,3,3,10
4,182,0,0,0
5,183,0,0,0
6,184,1,1,10
7,185,0,0,0
8,186,2,2,10
9,187,0,0,0


In [166]:
# Calcolo del punteggio massimo ottenibile per il Task 2 (solo se task1_predicted == 1)
df_merged["max_score_task2"] = 0
df_merged.loc[mask_abnormal, "max_score_task2"] = 10
df_merged.loc[(mask_abnormal) & (df_merged["Spacecraft No."] == 4), "max_score_task2"] = 20
max_task2_score = int(df_merged["max_score_task2"].sum())

task2_score = int(df_merged["score_task2"].sum())

print("Punteggio ottenuto sul Task 2: ", task2_score)
print("Punteggio massimo sul Task 2: ", max_task2_score)

Punteggio ottenuto sul Task 2:  320
Punteggio massimo sul Task 2:  340


### Task 3 evaluation

In [167]:
# --- Task 3: Bubble location (solo se task2 == 2) ---
mask_anomaly = df_merged["task2_predicted"] == 2
correct_task3 = (df_merged["task3_predicted"] == df_merged["task3_expected"]) & mask_anomaly
df_merged["score_task3"] = correct_task3.astype(int) * 10
df_merged.loc[df_merged["Spacecraft No."] == 4, "score_task3"] *= 2
df_merged[["Case", "task3_predicted","task3_expected", "score_task3"]]


,Case,task3_predicted,task3_expected,score_task3
0,178,2,2,10
1,179,0,0,0
2,180,0,0,0
3,181,0,0,0
4,182,0,0,0
5,183,0,0,0
6,184,0,0,0
7,185,0,0,0
8,186,6,6,10
9,187,0,0,0


In [168]:
# Calcolo del punteggio massimo ottenibile per il Task 3 (solo se task2_predicted == 2)
df_merged["max_score_task3"] = 0
df_merged.loc[mask_anomaly, "max_score_task3"] = 10
df_merged.loc[(mask_anomaly) & (df_merged["Spacecraft No."] == 4), "max_score_task3"] = 20

max_task3_score = int(df_merged["max_score_task3"].sum())

task3_score = int(df_merged["score_task3"].sum())

print("Punteggio ottenuto sul Task 3: ", task3_score)
print("Punteggio massimo sul Task 3: ", max_task3_score)

Punteggio ottenuto sul Task 3:  130
Punteggio massimo sul Task 3:  130


### Task 4 evaluation

In [169]:
# --- Task 4: Solenoid location (solo se task2 == 3) ---
mask_fault = df_merged["task2_predicted"] == 3
correct_task4 = (df_merged["task4_predicted"] == df_merged["task4_expected"]) & mask_fault
df_merged["score_task4"] = correct_task4.astype(int) * 10
df_merged.loc[df_merged["Spacecraft No."] == 4, "score_task4"] *= 2
df_merged[["Case", "task4_predicted","task4_expected", "score_task4"]]


,Case,task4_predicted,task4_expected,score_task4
0,178,0,0,0
1,179,2,2,10
2,180,0,0,0
3,181,4,4,10
4,182,0,0,0
5,183,0,0,0
6,184,0,0,0
7,185,0,0,0
8,186,0,0,0
9,187,0,0,0


In [170]:
# Calcolo del punteggio massimo ottenibile per il Task 4 (solo se task2_predicted == 3)
df_merged["max_score_task4"] = 0
df_merged.loc[mask_fault, "max_score_task4"] = 10
df_merged.loc[(mask_fault) & (df_merged["Spacecraft No."] == 4), "max_score_task4"] = 20

max_task4_score = int(df_merged["max_score_task4"].sum())

task4_score = int(df_merged["score_task4"].sum())

print("Punteggio ottenuto sul Task 4: ", task4_score)
print("Punteggio massimo sul Task 4: ", max_task4_score)

Punteggio ottenuto sul Task 4:  80
Punteggio massimo sul Task 4:  100


### Task 5 evaluation

In [171]:
# --- Task 5: Valve opening (solo se task2_predicted == 3) ---
df_merged["score_task5"] = 0

if "task5_predicted" in df_merged.columns:
    # Maschera per filtrare i casi dove task2_predicted è 3
    mask_task5 = df_merged["task2_predicted"] == 3

    # Calcolo dell'errore e del punteggio solo su quei casi
    error = np.abs(df_merged.loc[mask_task5, "task5_predicted"] - df_merged.loc[mask_task5, "task5_expected"])
    score_5 = (20 - error).clip(lower=0)

    # Inseriamo i punteggi calcolati solo per le righe corrette
    df_merged.loc[mask_task5, "score_task5"] = score_5

    # Raddoppiamo per gli spacecraft-4
    df_merged.loc[mask_task5 & (df_merged["Spacecraft No."] == 4), "score_task5"] *= 2

# Visualizzazione
df_merged[["Case", "task5_predicted", "task5_expected", "score_task5"]]

,Case,task5_predicted,task5_expected,score_task5
0,178,100,100,0
1,179,26,22,16
2,180,100,100,0
3,181,67,76,11
4,182,100,100,0
5,183,100,100,0
6,184,100,100,0
7,185,100,100,0
8,186,100,100,0
9,187,100,100,0


In [172]:
# Calcolo del punteggio massimo ottenibile per il Task 5 (solamente se task2_predicted == 3)
df_merged["max_score_task5"] = 0

if "task5_expected" in df_merged.columns:
    # Score massimo per ciascuna riga (senza errore -> 20)
    df_merged.loc[mask_fault, "max_score_task5"] = 20
    
    # Raddoppia se Spacecraft-4
    df_merged.loc[(mask_fault) & (df_merged["Spacecraft No."] == 4), "max_score_task5"] = 40

# Somma totale del punteggio massimo ottenibile
max_task5_score = int(df_merged["max_score_task5"].sum())

task5_score = int(df_merged["score_task5"].sum())

print("Punteggio ottenuto sul Task 5: ", task5_score)
print("Punteggio massimo sul Task 5: ", max_task5_score)


Punteggio ottenuto sul Task 5:  83
Punteggio massimo sul Task 5:  200


### Risultati finali

In [174]:
final_score = task1_score + task2_score + task3_score + task4_score + task5_score
max_final_score = max_task1_score + max_task2_score + max_task3_score + max_task4_score + max_task5_score

print("Punteggio finale ottenuto: ", final_score)
print("Punteggio massimo ottenibile: ", max_final_score)

print("\nPercentuale score: ", round((final_score/max_final_score)*100, 2), "%")


Punteggio finale ottenuto:  1253
Punteggio massimo ottenibile:  1460

Percentuale score:  85.82 %
